Copyright **`(c)`** 2023 Giovanni Squillero `<s316467@studenti.polito.it>`  
[`https://github.com/s316467/Computational-Intelligence-23-24`](https://github.com/s316467/Computational-Intelligence-23-24)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/s316467/Computational-Intelligence-23-24/LICENSE.md) for details.  

Initial code taken from previous lessons and prof. Squillero repo  

In [105]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [106]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [107]:
PROBLEM_SIZE = 20
NUM_SETS = 40
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

## Depth First

In [108]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.pop()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.append(new_state)
        current_state = frontier.pop()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

13it [00:00, 13007.15it/s]

Solved in 13 steps (13 tiles)


## Breadth First

In [109]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.pop()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.append(new_state)
        current_state = frontier.pop()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")


13it [00:00, 6502.02it/s]

Solved in 13 steps (13 tiles)


## Greedy Best First

In [110]:
def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [111]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

4it [00:00, 666.71it/s]

Solved in 4 steps (4 tiles)


## A*

In [112]:
"""
Prof final Heuristic
"""
def hProf(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

"""
My Heuristic
"""
def h(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_elements = PROBLEM_SIZE - np.sum(already_covered)
    average_set_size = np.mean([np.sum(s) for s in SETS])
    return missing_elements / average_set_size

def fProf(state):
    return len(state.taken) + hProf(state)

def f(state):
    return len(state.taken) + h(state)


Comparison between prof's heuristic (first result) and mine (second).

The main goal is to minmize the number of tiles used to cover the set while trying to minimizing the amount of steps taken

In [113]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((fProf(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

55it [00:02, 21.14it/s]

Solved in 55 steps (4 tiles)


In [114]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

29it [00:00, 201.39it/s]

Solved in 29 steps (4 tiles)


The goal of a heuristic in A* search is to provide an estimate of the cost to reach the goal from a given state. An ideal heuristic should be both fast to compute and effective in guiding the search towards the goal.

We have a set of elements (`PROBLEM_SIZE`) and a collection of sets (`SETS`), and we want to find the minimum number of sets that cover all elements.

A good heuristic could be the number of uncovered elements divided by the average size of the sets. This would give an estimate of the minimum number of sets needed to cover all remaining elements.

This heuristic is both fast to compute and should provide a good estimate of the cost to reach the goal. The average set size gives us an estimate of how many elements each set can cover, so dividing the number of missing elements by the average set size gives us an estimate of how many sets we still need.